In [71]:
import pandas as pd
import numpy as np
import nbimporter
import feature_engineering
from datetime import datetime, timedelta
import glob
import os
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [72]:
temp_path =r"C:\Users\Isabell\EMSI\emsi_ml\Wetter\Berlin\Temperature"
temp_file = glob.glob(os.path.join(temp_path, "produkt*"))

temperature_dic = {}
years_of_interest = [2012, 2014, 2016, 2018, 2021]

for file in temp_file:
    temperature = pd.read_csv(file, delimiter = ";")
    temperature["MESS_DATUM"] = pd.to_datetime(temperature["MESS_DATUM"], format='%Y%m%d%H%M')
    #print(temperature)
    for year in years_of_interest:
        #print(temperature)
        data = temperature[temperature["MESS_DATUM"].dt.year == year]
        data.set_index("MESS_DATUM", inplace = True)
        
        #print(data)
        if not data.empty:
            temperature_dic[year] =  temperature[temperature["MESS_DATUM"].dt.year == year]
    
print(temperature_dic[2014])

        STATIONS_ID          MESS_DATUM    QN   PP_10  TT_10  TM5_10  RF_10  \
210384          433 2014-01-01 00:00:00     3  1009.1   -0.1    -1.4   94.8   
210385          433 2014-01-01 00:10:00     3  1009.1   -0.1    -1.6   94.6   
210386          433 2014-01-01 00:20:00     3  1009.0   -0.2    -1.5   94.3   
210387          433 2014-01-01 00:30:00     3  1009.0   -0.2    -1.6   94.2   
210388          433 2014-01-01 00:40:00     3  1009.0   -0.3    -1.8   94.1   
...             ...                 ...   ...     ...    ...     ...    ...   
262939          433 2014-12-31 23:10:00     3  1026.1    5.1     4.5   96.8   
262940          433 2014-12-31 23:20:00     3  1026.1    5.2     4.6   96.3   
262941          433 2014-12-31 23:30:00     3  1026.1    5.2     4.5   96.0   
262942          433 2014-12-31 23:40:00     3  1026.1    5.2     4.6   95.7   
262943          433 2014-12-31 23:50:00     3  1026.1    5.2     4.6   95.3   

        TD_10  eor  
210384   -0.8  eor  
210385   

In [73]:
folder_path = r"Stromverbrauch\Viertel"
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

stromverbrauch_list = []

for file in csv_files:
    #folder_path = r"C:\Users\Isabell\EMSI\emsi_ml\Stromverbrauch\Viertel\Realisierter_Stromverbrauch_201506070000_201506100000_Viertelstunde.csv"
    stromverbrauch = pd.read_csv(file, delimiter = ";")
    stromverbrauch["Datum von"] = pd.to_datetime(stromverbrauch["Datum von"], format='%d.%m.%Y %H:%M')
    stromverbrauch_list.append(stromverbrauch)


df_tuples = [(stromverbrauch["Datum von"].iloc[0], stromverbrauch) for stromverbrauch in stromverbrauch_list ]

# Sort the list of tuples by the earliest date
df_tuples_sorted = sorted(df_tuples, key=lambda x: x[0])

dfs_by_year = {}

# Iterate over the list of tuples
for dt, df in df_tuples_sorted:  # Correctly unpack: dt is the date, df is the DataFrame
    year = dt.year  # Extract the year from the date
    
    if year not in dfs_by_year:
        dfs_by_year[year] = df
    else:
        # Merge the DataFrame with the existing one for the same year
        dfs_by_year[year] = pd.concat([dfs_by_year[year], df], ignore_index=True)

# Extract the sorted DataFrames into a list
sorted_df_list = [df_tuple[1] for df_tuple in df_tuples_sorted]

for year, merged_df in dfs_by_year.items():
    print(f"Year: {year}")
    print(merged_df)



Year: 2015
               Datum von         Datum bis  \
0    2015-06-07 00:00:00  07.06.2015 00:15   
1    2015-06-07 00:15:00  07.06.2015 00:30   
2    2015-06-07 00:30:00  07.06.2015 00:45   
3    2015-06-07 00:45:00  07.06.2015 01:00   
4    2015-06-07 01:00:00  07.06.2015 01:15   
...                  ...               ...   
3163 2015-07-12 22:45:00  12.07.2015 23:00   
3164 2015-07-12 23:00:00  12.07.2015 23:15   
3165 2015-07-12 23:15:00  12.07.2015 23:30   
3166 2015-07-12 23:30:00  12.07.2015 23:45   
3167 2015-07-12 23:45:00  13.07.2015 00:00   

     Gesamt (Netzlast) [MWh] Originalauflösungen  
0                                      10.419,00  
1                                      10.234,50  
2                                      10.000,00  
3                                       9.836,75  
4                                       9.703,00  
...                                          ...  
3163                                   11.883,50  
3164                        

In [74]:
folder_path = r"Spielplan"
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

spielplan_list = []

for file in csv_files:
    spielplan = pd.read_csv(file, delimiter = ";")
    spielplan["Land1"] = spielplan["Land1"].str.strip()
    spielplan["Land2"] = spielplan["Land2"].str.strip()
    spielplan["DateTime"] = spielplan["Datum"] + " " + spielplan["Uhrzeit (MESZ)"]
    try:
        spielplan["DateTime"] = pd.to_datetime(spielplan["DateTime"], format='%d. %b %y %H:%M')
    except:
        spielplan["DateTime"] = pd.to_datetime(spielplan["DateTime"], format='%d.%m.%Y %H:%M')

    if "EM" in file:
        spielplan["Liga"] = "EM"
    elif "WM" in file:
        spielplan["Liga"] = "WM"



    #print(spielplan.columns)
    #spielplan["Datum"] = pd.to_datetime(spielplan["Datum"], format = "%d. %b %y")

   
    #spielplan.set_index("DateTime", inplace = True)

    spielplan = spielplan.loc[:, ["Land1", "Land2", "Runde", "Liga", "DateTime"]]
    #print(spielplan)
    grouped = spielplan.groupby("DateTime")
    #print(grouped)
    # Initialize a new DataFrame to hold the results
    results = pd.DataFrame()
    
    #row = {}
    for name, group in grouped:
        #print(name)
        #print(group)
        if len(group) == 1:
            results = pd.concat([results, group])
            #print(results)
            #results["DateTime"] = name
            #results["Land3"] = np.nan
            #results["Land4"] = np.nan
        else:
            land1_value = group.iloc[0]['Land1']
            land2_value = group.iloc[0]['Land2']
            
            # Update results DataFrame (Example: adding values from the second row to a new DataFrame)
            new_row = {
                "DateTime" : name,
                'Land1': land1_value,
                'Land2': land2_value,
                "Runde" : group.iloc[1]["Runde"],
                "Liga": group.iloc[1]["Liga"],
                'Land3': group.iloc[1]['Land1'],
                'Land4': group.iloc[1]['Land2'],
                
            }

            new_df = pd.DataFrame([new_row])
            #print(new_df)
            #new_df.set_index("DateTime")
            results = pd.concat([results, new_df], ignore_index = True)
    results.set_index("DateTime")
    
    spielplan_list.append(results)

        #print(results["Runde"].unique())
        #print(results)
#print(lenspielplan_list)
print(len(spielplan_list))
    

5


In [75]:

#index_counter = 0
counter_limit = len(spielplan_list) - 1

merged_df_dic = {}

print(dfs_by_year.keys())

for key,value in dfs_by_year.items():
    print(key)
    found_item = False
    index_counter = 0
    while found_item == False:
        spielplan = spielplan_list[index_counter]
        #rint(spielplan)
        spielplan_name_list = spielplan.columns.tolist()

        if key == spielplan["DateTime"].iloc[0].year:
            merged_df = pd.merge(value, spielplan, left_on="Datum von", right_on='DateTime', how='outer')
            
            for index, row in spielplan.iterrows():
                start_time = row["DateTime"]
                
                until_time = row["DateTime"] + timedelta(minutes=105)
                #print(until_time)
                fill_condition = (merged_df['Datum von']>= start_time) & (merged_df['Datum von'] <= until_time)
                merged_df.loc[fill_condition,spielplan_name_list] = merged_df.loc[fill_condition, spielplan_name_list].fillna(method='ffill')
                print(merged_df)
            if not merged_df.empty:
                #print(year)
                merged_df_dic[key] = merged_df
            #merged_df_list.append(merged_df)
            found_item = True
        elif index_counter == counter_limit:
            found_item = True
        else:
            index_counter += 1
    
#df = len(merged_df_list)

#df = df.loc[2965:2980,["Datum von", "Land1", "Gesamt (Netzlast) [MWh] Originalauflösungen"]]

print(merged_df_dic.keys())

#print(df["Land1"].unique())
#df.set_index("Datum von", inplace = True)
#data = df.loc["2016-06-11"]            
#print(data)
#merged_df_list.append(merged_df)

dict_keys([2015, 2016, 2017, 2018, 2019, 2021, 2023])
2015
2016
               Datum von         Datum bis  \
0    2016-06-10 00:00:00  10.06.2016 00:15   
1    2016-06-10 00:15:00  10.06.2016 00:30   
2    2016-06-10 00:30:00  10.06.2016 00:45   
3    2016-06-10 00:45:00  10.06.2016 01:00   
4    2016-06-10 01:00:00  10.06.2016 01:15   
...                  ...               ...   
2971 2016-07-10 22:45:00  10.07.2016 23:00   
2972 2016-07-10 23:00:00  10.07.2016 23:15   
2973 2016-07-10 23:15:00  10.07.2016 23:30   
2974 2016-07-10 23:30:00  10.07.2016 23:45   
2975 2016-07-10 23:45:00  11.07.2016 00:00   

     Gesamt (Netzlast) [MWh] Originalauflösungen Land1 Land2 Runde Liga  \
0                                      11.930,50   NaN   NaN   NaN  NaN   
1                                      11.764,50   NaN   NaN   NaN  NaN   
2                                      11.602,50   NaN   NaN   NaN  NaN   
3                                      11.495,00   NaN   NaN   NaN  NaN   
4       

In [78]:
merged_dic_final = {}
for key,value in merged_df_dic.items():
    #value.drop(columns = ["level_0"], inplace = True)
    temp_df = temperature_dic[key]
    temp_df.reset_index(inplace = True)
    #print(temp_df)
    temp_df.rename(columns={"MESS_DATUM": "DateTime"}, inplace = True)
    #print(value)
    temp_df.set_index("DateTime", inplace=True)

    numeric_cols = temp_df.select_dtypes(include='number').columns
    df_resampled_numeric = temp_df[numeric_cols].resample('15T').mean()

    # Resample non-numeric columns
    non_numeric_cols = temp_df.select_dtypes(exclude='number').columns
    df_resampled_non_numeric = temp_df[non_numeric_cols].resample('15T').agg(lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0])

    # Combine the resampled numeric and non-numeric dataframes
    df_resampled = pd.concat([df_resampled_numeric, df_resampled_non_numeric], axis=1)
    
    #temp_df_resampled = temp_df.resample("15T").mean()

    df_resampled.reset_index(inplace = True)
    value.reset_index(inplace = True)

    merged_df_final = pd.merge(value, df_resampled, left_on="Datum von", right_on='DateTime', how = "left")

    if not df_resampled.empty:
        merged_dic_final[key] = merged_df_final
        
print(merged_dic_final.keys())

C:\Users\Isabell\AppData\Local\Temp\ipykernel_1476\3495137154.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.rename(columns={"MESS_DATUM": "DateTime"}, inplace = True)


dict_keys([2016, 2018, 2021])
